In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import textblob
import gensim
import spacy
import os

import data
import plot

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Takes a 5 minutes the first time, but use `use_cached` afterward so you don't have to wait in the future.

In [5]:
train, train_vecs, test, test_vecs = data.get_data(use_cached=True)

train['utter_len'] = train['utter'].map(lambda x: len(x.split(' ')))

train['has_question_mark'] = train['utter'].map(lambda x: 1 if'?' in x.split(' ') else 0)
train = pd.concat([train, data.get_lags(train[['has_question_mark']], lag_range=[1])], axis=1)

Using Cached


In [6]:
LAG_MASK = ['act', 'emo', 'polarity', 'subjectivity']
train_lags = data.get_lags(train[LAG_MASK], lag_range=[1, 2])

Is first utterance?

### Classify Dialogue Acts

In [7]:
from collections import OrderedDict
from cycler import cycler

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import precision_recall_curve, confusion_matrix

from chatty.utils import dd_encodings

ACT_MASK = train['act'].notnull() & train['has_question_mark_t-1'].notnull()

assert train['act'][train['act'].notnull()].shape[0] \
      == train_vecs[train['act'].notnull()].shape[0]

X = np.concatenate((train_vecs,
                    train['first_utterance'].values.reshape(-1, 1),
                    train['subjectivity'].values.reshape(-1, 1),
                    train['utter_len'].values.reshape(-1, 1),
                    train['has_question_mark'].values.reshape(-1, 1)), axis=1)[ACT_MASK]
y = train['act'][ACT_MASK]

In [129]:
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import confusion_matrix

from sklearn.linear_model.logistic import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [10]:
results = data.cv_stratified_shuffle(X, y,
                                     RandomForestClassifier(n_jobs=-1),
                                     splits=5)
plot.plot_prec_rec(results)

Training
Training
Training
Training
Training


In [139]:
%reload_ext plot

In [167]:
title = "RandomForest"
plot.plot_prec_rec(results, title, fig=fig)
le = LabelEncoder()
le.fit(results['classes'])
cm = confusion_matrix(le.transform(np.concatenate(results['y_true'])),
                      np.argmax(np.concatenate(results['y_proba']), axis=1))
plt.figure()
plot.plot_confusion_matrix(cm, le.classes_, normalize=True, title=title);

AttributeError: 'tuple' object has no attribute 'set_prop_cycle'